In [1]:
import numpy as np
from scipy import io
from spdnet_init_afew import spdnet_init_afew
from spdnet.utils.load_data import load_data
from accumulate_gradients import accumulate_gradients

from os import listdir
import os

from sklearn.utils import shuffle

# spdnet_afew

In [2]:
opts = {}

opts['batchSize'] = 50
opts['test_batchSize'] = 1
opts['numEpochs'] = 5
opts['learningRate'] = 0.001 * np.ones(opts['numEpochs'])
opts['weightDecay'] = 0.05
opts['continue'] = 1


#  Инициализация

# load_data

In [3]:
#train_data, train_labels = load_data('train')
#val_data, val_labels = load_data('val')

In [4]:
#print(train_data.shape, train_labels.shape, val_data.shape, val_labels.shape)

In [5]:
#X_train, y_train = shuffle(train_data, train_labels, random_state=123)
#X_val, y_val = shuffle(val_data, val_labels, random_state=123)

In [6]:
#np.save('X_train', X_train)
#np.save('X_val', X_val)
#np.save('y_train', y_train)
#np.save('y_val', y_val)

In [7]:
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

In [8]:
def iterate_minibathces(X, y, batchsize):
    """
    X = 3-dim tensor (data_size, rows=400, cols=400)
    """
    indices = np.random.permutation(np.arange(X.shape[0]))
    for start in range(0, len(indices), batchsize):
        ix = indices[start: start + batchsize]
        yield X[ix], y[ix]

In [9]:
from spdnet.vl_folder.vl_myforbackward import vl_myforbackward

In [10]:
def calculate_error(true_labels, predictions):
    
    
    #print(true_labels)
    ind = np.argsort(-predictions, axis = 1)[:, 0]
    print(ind)
    answer = np.sum(ind == true_labels - 1)
    
    return answer / predictions.shape[0]

In [11]:
# данные имеют следующие размерности: [batch_size, width, height]
# после размножения: [batch_size, n_channels, width, height]

net = spdnet_init_afew()

for epoch in range(opts['numEpochs']):
    # ставим сеть в режим обучения.
    #net.train(True)
    
    training_error = []
    validation_error = []
    
    

    zz = 0
    
    net['train_mode'] = True
    
    for X_batch, y_batch in iterate_minibathces(X_train, y_train, opts['batchSize']):
        print('batch num: ', zz + 1)
        # net.layers{end}.class = spd_label
        #learningRate = opts['learningRate'][epoch]
        
        net['layers'][-1]['class'] = y_batch
        res = vl_myforbackward(net=net, x=X_batch, dzdy=1, res=None, varargin=None)
        [net, res] = accumulate_gradients(opts=None, lr=opts['learningRate'][epoch], 
                                          batchSize=X_batch.shape[0], net=net, res=res)
        predictions = res['x'][-2]
        #print(len(res['x']))
        
        training_error.append(calculate_error(y_batch, predictions))
        
        zz += 1
    
    # ставим сеть в режим оценки ошибки
    #net.train(False)
    
    net['train_mode'] = False
    
    for X_batch, y_batch in iterate_minibathces(X_val, y_val, opts['batchSize']):
        net['layers'][-1]['class'] = y_batch
        res = vl_myforbackward(net=net, x=X_batch, dzdy=None, res=None)
        [net, res] = accumulate_gradients(opts=None, lr=opts['learningRate'][epoch], 
                                          batchSize=X_batch.shape[0], net=net, res=res)
        
        
        predictions = res['x'][-2]
        
        validation_error.append(calculate_error(y_batch, predictions))
        
        
    
    print("Number of epoch: ", epoch)
    print("Training error: ", np.mean(np.array(training_error)))
    print("Validation error: ", np.mean(np.array(validation_error)))

batch num:  1
[6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6]
batch num:  2
[4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 3 4 4 4 4 3 4 4 4 4 4 4 4 4
 4 4 4 4 3 4 4 4 4 4 4 4 4]
batch num:  3
[3 3 4 3 3 3 3 4 3 4 3 3 3 3 3 3 4 4 3 3 3 3 3 3 3 3 4 4 3 3 4 3 3 4 3 3 3
 4 3 3 4 4 3 3 3 4 3 3 4 3]
batch num:  4
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3]
batch num:  5
[3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 4 3 3 3 3 3 3 3 3 3 3 3 3]
batch num:  6
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4]
batch num:  7
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4]
batch num:  8
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4]
batch num:  9
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4

ValueError: operands could not be broadcast together with shapes (50,100) (100,50) 

In [12]:
import scipy

In [13]:
scipy.__version__

'0.17.1'